Currently working.
* One adjustment to make in the future: Require location to include the state if you are doing a county search in to_timeseries(). There are multiple counties with the same name residing in different states. Fortunately, if you're interested in Connecticut, this tends to alphabetically pop up early in the dataframe.

In [1]:
import pandas as pd
import csv
import us
import matplotlib # for testing
%matplotlib inline

# A list of all the URLs to state and county newdx data sets
raw_newdx_urls = {
    2015: {"state":"https://github.com/jamiekasulis/aidsvu_data_grab/raw/master/AIDSVu_State_NewDX_2015.xlsx",
        "county":"https://github.com/jamiekasulis/aidsvu_data_grab/raw/master/AIDSVu_County_NewDX_2015.xlsx"},
    2014: {"state":"https://github.com/jamiekasulis/aidsvu_data_grab/raw/master/AIDSVu_State_NewDX_2014.xlsx",
        "county":"https://github.com/jamiekasulis/aidsvu_data_grab/raw/master/AIDSVu_County_NewDX_2014.xlsx"},
    2013: {"state":"https://github.com/jamiekasulis/aidsvu_data_grab/raw/master/AIDSVu_State_NewDX_2013.xlsx",
        "county":"https://github.com/jamiekasulis/aidsvu_data_grab/raw/master/AIDSVu_County_NewDX_2013.xlsx"},
    2012: {"state":"https://github.com/jamiekasulis/aidsvu_data_grab/raw/master/AIDSVu_State_NewDX_2012.xlsx",
        "county":"https://github.com/jamiekasulis/aidsvu_data_grab/raw/master/AIDSVu_County_NewDX_2012.xlsx"},
    2011: {"state":"https://github.com/jamiekasulis/aidsvu_data_grab/raw/master/AIDSVu_State_NewDX_2011.xlsx",
        "county":"https://github.com/jamiekasulis/aidsvu_data_grab/raw/master/AIDSVu_County_NewDX_2011.xlsx"},
    2010: {"state":"https://github.com/jamiekasulis/aidsvu_data_grab/raw/master/AIDSVu_State_NewDX_2010.xlsx",
        "county":"https://github.com/jamiekasulis/aidsvu_data_grab/raw/master/AIDSVu_County_NewDX_2010.xlsx"},
    2009: {"state":"https://github.com/jamiekasulis/aidsvu_data_grab/raw/master/AIDSVu_State_NewDX_2009.xlsx",
        "county":"https://github.com/jamiekasulis/aidsvu_data_grab/raw/master/AIDSVu_County_NewDX_2009.xlsx"},
    2008: {"state":"https://github.com/jamiekasulis/aidsvu_data_grab/raw/master/AIDSVu_State_NewDX_2008.xlsx",
        "county":"https://github.com/jamiekasulis/aidsvu_data_grab/raw/master/AIDSVu_County_NewDX_2008.xlsx"}
}

# A list of all the URLs to state and county prev data sets.
# Note that there is no prevalence data for 2008, 2009, or 2015.
raw_prev_urls = {
    2014: {"state":"https://github.com/jamiekasulis/aidsvu_data_grab/raw/master/AIDSVu_State_Prev_2014.xlsx",
        "county":"https://github.com/jamiekasulis/aidsvu_data_grab/raw/master/AIDSVu_County_Prev_2014.xlsx"},
    2013: {"state":"https://github.com/jamiekasulis/aidsvu_data_grab/raw/master/AIDSVu_State_Prev_2013.xlsx",
        "county":"https://github.com/jamiekasulis/aidsvu_data_grab/raw/master/AIDSVu_County_Prev_2013v2.xlsx"},
    2012: {"state":"https://github.com/jamiekasulis/aidsvu_data_grab/raw/master/AIDSVu_State_2012-2-1.xlsx",
        "county":"https://github.com/jamiekasulis/aidsvu_data_grab/raw/master/AIDSVu_County_2012-1.xls"},
    2011: {"state":"https://github.com/jamiekasulis/aidsvu_data_grab/raw/master/AIDSVu_State_20111.xlsx",
        "county":"https://github.com/jamiekasulis/aidsvu_data_grab/raw/master/AIDSVu_County_2011-1-1.xlsx"},
    2010: {"state":"https://github.com/jamiekasulis/aidsvu_data_grab/raw/master/AIDSVu_State_2010-1.xls",
        "county":"https://github.com/jamiekasulis/aidsvu_data_grab/raw/master/AIDSVu_County_20101.xls"}
}

In [2]:
def make_raw_df(year, sc, dataset):
    """
    Returns a raw, uncleaned dataframe corresponding to year and state or county in either raw_newdx_urls or raw_prev_urls.
    year is a full four-digit year int, sc is the string "state" or "county", and dataset is the string "prev" or "newdx".
    """
    if dataset == "newdx":
        return pd.read_excel(raw_newdx_urls.get(year).get(sc), skiprows=2)
    elif dataset == "prev":
        if year == 2012:
            # 2012 is a special case where you should not skip any rows.
            return pd.read_excel(raw_prev_urls.get(year).get(sc), skiprows=0)
        elif year == 2011:
            return pd.read_excel(raw_prev_urls.get(year).get(sc), skiprows=0)
        elif year == 2010:
            return pd.read_excel(raw_prev_urls.get(year).get(sc), skiprows=0)
        else:
            return pd.read_excel(raw_prev_urls.get(year).get(sc), skiprows=2)
    else:
        return None

In [3]:
def omit_us_territories(countydf):
    """
    Takes a county data frame and returns a version of it that only includes the 50 U.S. states.
    This is a helper function for clean_newdx_dfs()
    """
    if 'GEO ID' in countydf.columns:
        countydf = countydf[countydf['GEO ID'].astype('int64') < 60000] # U.S. territories/non-states have geo IDs 60000 and above
    return countydf

In [225]:
def convert_abbreviations(df):
    """
    Checks to see if the values in the 'state' column are not full state names, but abbreviations.
    If that it is the case, it will convert each abbreviation to the full state name.
    """
    us_state_abbrev = {
        'Alabama': 'AL', 'Alaska': 'AK', 'Arizona': 'AZ', 'Arkansas': 'AR', 'California': 'CA', 'Colorado': 'CO',
        'Connecticut': 'CT','Delaware': 'DE','Florida': 'FL','Georgia': 'GA','Hawaii': 'HI','Idaho': 'ID','Illinois': 'IL',
        'Indiana': 'IN','Iowa': 'IA','Kansas': 'KS','Kentucky': 'KY','Louisiana': 'LA','Maine': 'ME','Maryland': 'MD',
        'Massachusetts': 'MA','Michigan': 'MI','Minnesota': 'MN','Mississippi': 'MS','Missouri': 'MO','Montana': 'MT',
        'Nebraska': 'NE','Nevada': 'NV','New Hampshire': 'NH','New Jersey': 'NJ','New Mexico': 'NM','New York': 'NY',
        'North Carolina': 'NC','North Dakota': 'ND','Ohio': 'OH','Oklahoma': 'OK','Oregon': 'OR','Pennsylvania': 'PA',
        'Rhode Island': 'RI','South Carolina': 'SC','South Dakota': 'SD','Tennessee': 'TN','Texas': 'TX',
        'Utah': 'UT','Vermont': 'VT','Virginia': 'VA','Washington': 'WA','West Virginia': 'WV','Wisconsin': 'WI',
        'Wyoming': 'WY', 'Washington, DC' : 'DC', 'Puerto Rico' : 'PR'
    }
    # Make the keys the values, and vice versa.
    us_state_abbrev = { val:key for (key, val) in us_state_abbrev.items() }
    # Convert every state in the column
    for s in range(len(df['State'])):
        abbr = df['State'].iloc[s]
        if type(abbr) == str:
            if len(abbr) > 2:
                break
            else:
                 df['State'].replace([abbr], us_state_abbrev[abbr], inplace=True)
        else:
            break
    
    return df

In [226]:
# make_raw_df for all newdx data and store in newdx_dfs
def make_clean_newdx_dfs():
    """
    Makes and returns a dictionary of newdx data indexed by year and state/county.
    County data will have their U.S. territories removed.
    """
    newdx_dfs = {}
    for year in raw_newdx_urls.keys():
        # make the data frames
        # strip columns and set them to lower case
        newdx_dfs[year] = {"state": convert_abbreviations(make_raw_df(year, 'state', 'newdx')).rename(columns=lambda x: x.strip().lower()),
                       "county": convert_abbreviations(make_raw_df(year, 'county', 'newdx')).rename(columns=lambda x: x.strip().lower())}
        
    return newdx_dfs

In [227]:
# make_raw_df for all prev data and store in prev_dfs
def make_clean_prev_dfs():
    """
    Makes and returns a dictionary of prevalence data indexed by year and state/county.
    County data will have their U.S. territories removed.
    """
    prev_dfs = {}
    for year in raw_prev_urls.keys():
        prev_dfs[year] = {"state": convert_abbreviations(make_raw_df(year, 'state', 'prev')).rename(columns=lambda x: x.strip().lower()),
                       "county": convert_abbreviations(omit_us_territories(make_raw_df(year, 'county', 'prev'))).rename(columns=lambda x: x.strip().lower())}

    return prev_dfs

In [228]:
prev_dfs = make_clean_prev_dfs()

In [229]:
newdx_dfs = make_clean_newdx_dfs()

In [230]:
def state_or_county(loc_name):
    """
    A helper function for to_timeseries. Given the name of a location, this function
    will return 'county' or 'city' based on whether loc_name ends with 'county' or not.
    """
    if 'county' in loc_name.lower():
        return 'county' # return 'county name' and not 'county' because this is the name of the column in county data
    else:
        return 'state'

In [231]:
def determine_proper_dataset(column_name, loc_type):
    """
    A helper function for determine_proper_years() and to_timeseries().
    Returns either newdx_dfs or prev_dfs, whichever one has a column called column_name.
    loc_type should be either 'city' or 'county'
    """
    # Check newdx_dfs first.
    for year in newdx_dfs.keys():
        df = newdx_dfs[year][loc_type]
        df_cols = df.columns.tolist()
        if column_name in df_cols:
            return newdx_dfs
    
    # Check prev_dfs
    for year in prev_dfs.keys():
        df = prev_dfs[year][loc_type]
        df_cols = df.columns.tolist()
        if column_name in df_cols:
            return prev_dfs
        

    print("ERROR: column_name does not exist in newdx_dfs or prev_dfs based on argument loc_type (county or city).")
    return None

In [232]:
# Determine the proper start and end years to build the time series on.
# If start_year is not a key in the dictionary of data frames, do the closest year working DOWN
# For end_year, do the closest year working UP.
def determine_proper_years(start_year, end_year, data_dict):
    """
    A helper function for to_timeseries. It verifies that data_dict has data for start_year and end_year.
    If not, it will return the minimum start_year and maximum end_year to make a time series as wide as possible.
    Returns a list [start, end]
    data_dict should be prev_dfs or newdx_dfs.
    """
    start = None
    end = None
    
    if start_year in data_dict.keys():
        start = start_year
    else:
        start = min(list(data_dict.keys()))
    if end_year in data_dict.keys():
        end = end_year
    else:
        end = max(list(data_dict.keys()))

    return [start, end]

In [233]:
def get_valid_years(year_range, data_dict):
    """
    A helper function for get_timeseries.
    Given a 2-element list [start_year, end_year] and a data_dict that is either newdx_dfs or prev_dfs,
    return an ordered list (ascending) of years that are actually in data_dict.
    This function is meant to account for the fact that there may be missing years of data.
    """
    start = year_range[0]
    end = year_range[1]
    
    years = list(range(start, end+1)) # all the years in year_range
    proper_years = [] # ordered list of the years in year_range that we have data for
    
    # add the years that we have data for to proper_years
    keys = data_dict.keys()
    for year in years:
        if year in keys:
            proper_years.append(year)
    
    # impose ascending order on proper_years
    proper_years.sort()
    return proper_years

In [255]:
def to_timeseries(column_name, location, loc_type, start_year=2008, end_year=2014):
    """
    Returns a 2D table (year x value) going from start_year to end_year, where the values are from whichever column in
    either newdx or prev data that matches the column_name argument.
    The argument data_set should be either "prev" or "newdx". Defaults to newdx if no value is passed.
    Defaults to 2008 and 2014 for start and end years.
    """
    # Set loc_header
    if loc_type == 'state':
        loc_header = 'state'
    else:
        loc_header = 'county name'
    
    # Grab the dictionary of data frames that we should be looking for column_name in
    data_dict = determine_proper_dataset(column_name, loc_type)
    
    # Verify that the time range is valid
    time_range = determine_proper_years(start_year, end_year, data_dict)
    
    # Make an ordered list of the valid years in the time range (there may be holes), which we will iterate through later
    data_years = get_valid_years(time_range, data_dict)
    
    # Create the time series.
    time_series = pd.DataFrame(columns = ['year', column_name])
    
    # Iterate through every year in data_years, grabbing the value for column_name and storing it to the dataframe
    # called time_series.
    for dy in data_years:
        current_df = data_dict[dy][loc_type]
        value = current_df[current_df[loc_header] == location][column_name].iloc[0]
        time_series = time_series.append({'year': dy, column_name: value}, ignore_index=True)
    
    return time_series[['year', column_name]]

In [256]:
# Test: state prev
to_timeseries('black rate', 'Connecticut', 'state', 2010, 2014)

,year,black rate
0,2010.0,1270.5
1,2011.0,1129.0
2,2012.0,1167.0
3,2013.0,1144.0
4,2014.0,1149.0


In [257]:
to_timeseries('new diagnoses state rate', 'Connecticut', 'state', 2008, 2014)

,year,new diagnoses state rate
0,2008,12
1,2009,12
2,2010,13
3,2011,12
4,2012,10
5,2013,11
6,2014,10


In [260]:
# Test: state newdx
to_timeseries('new diagnoses black rate', 'Connecticut', 'state', 2008, 2018)

,year,new diagnoses black rate
0,2008,51
1,2009,56
2,2010,53
3,2011,47
4,2012,42
5,2013,44
6,2014,46
7,2015,33


In [261]:
# Test: county newdx
newdx_dfs[2010]['county'].columns
to_timeseries('new diagnoses rate', 'Fairfield County', 'county', 2008, 2018)

,year,new diagnoses rate
0,2008,15
1,2009,14
2,2010,14
3,2011,13
4,2012,10
5,2013,12
6,2014,13
7,2015,8
